# k means ||

In [1]:
# import the python libraries to create/connect to a Spark Session
from pyspark.sql import SparkSession

# build a SparkSession 
#   connect to the master node on the port where the master node is listening (7077)
#   declare the app name 
#   configure the executor memory to 512 MB
#   either *connect* or *create* a new Spark Context
spark = SparkSession.builder \
    .master("spark://spark-master:7077")\
    .appName("prova iniziale")\
    .config("spark.executor.memory", "512m")\
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/08/25 13:59:14 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Check the content of the Spark Session `spark` object.

This is the entry point to the main Spark functionalities.

In [2]:
spark

The __Spark UI__ link provided won't work, as it refers to the Docker container running the Spark application.

Port mapping is however provided for the `4040` port and you can open the Spark UI link to the application pointing your browser to `localhost:4040`.

From the Spark Session we can access the Spark Context.

The Spark Context is the driver application program we will use to submit applications to Spark, and it is used to work with RDDs

In [3]:
# create a spark context
sc = spark.sparkContext

# print its status
sc

<SparkContext master=spark://spark-master:7077 appName=prova iniziale>

## Import data from sklearn



In [4]:
# !pip install scikit-learn # to be run at the launch of "docker compose up"

In [5]:
import numpy as np
import pandas as pd
import sklearn.datasets #va installato

prova =  sklearn.datasets.fetch_kddcup99(percent10 = True, as_frame = True)
X = prova.data

In [6]:
# Show the dataframe (to be deleted for a bigger dataframe) (TO BE DELETED)
X

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate
0,0,b'tcp',b'http',b'SF',181,5450,0,0,0,0,...,9,9,1.0,0.0,0.11,0.0,0.0,0.0,0.0,0.0
1,0,b'tcp',b'http',b'SF',239,486,0,0,0,0,...,19,19,1.0,0.0,0.05,0.0,0.0,0.0,0.0,0.0
2,0,b'tcp',b'http',b'SF',235,1337,0,0,0,0,...,29,29,1.0,0.0,0.03,0.0,0.0,0.0,0.0,0.0
3,0,b'tcp',b'http',b'SF',219,1337,0,0,0,0,...,39,39,1.0,0.0,0.03,0.0,0.0,0.0,0.0,0.0
4,0,b'tcp',b'http',b'SF',217,2032,0,0,0,0,...,49,49,1.0,0.0,0.02,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
494016,0,b'tcp',b'http',b'SF',310,1881,0,0,0,0,...,86,255,1.0,0.0,0.01,0.05,0.0,0.01,0.0,0.0
494017,0,b'tcp',b'http',b'SF',282,2286,0,0,0,0,...,6,255,1.0,0.0,0.17,0.05,0.0,0.01,0.0,0.0
494018,0,b'tcp',b'http',b'SF',203,1200,0,0,0,0,...,16,255,1.0,0.0,0.06,0.05,0.06,0.01,0.0,0.0
494019,0,b'tcp',b'http',b'SF',291,1200,0,0,0,0,...,26,255,1.0,0.0,0.04,0.05,0.04,0.01,0.0,0.0


In [7]:
# Functions to be used in PANDAS that MUST NOT be used in this task

# Y = prova.target
# np.unique(Y)
# np.unique(X["flag"])
# np.unique(X["service"])
# np.unique(X["protocol_type"])
# np.shape(X)
# X.loc[100]

## Create a Spark dataframe
And then acquire important informations about the dataframe: # columns, # rows, # partitions, the schema, ...

In [8]:
# Create the spark dataframe with a smaller chunk of data (just to work easily)
import random
X_smaller = X.iloc[random.sample(range(1, len(X.index)), 1000)]
spark_X = spark.createDataFrame(X_smaller)

/usr/bin/spark-3.3.2-bin-hadoop3/python/pyspark/sql/pandas/conversion.py:474: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():
/usr/bin/spark-3.3.2-bin-hadoop3/python/pyspark/sql/pandas/conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


In [9]:
X_smaller

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate
196003,0,b'icmp',b'ecr_i',b'SF',1032,0,0,0,0,0,...,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
39889,0,b'tcp',b'http',b'SF',54540,8314,0,0,0,2,...,167,167,1.0,0.0,0.01,0.0,0.0,0.0,0.05,0.05
12406,0,b'tcp',b'http',b'SF',294,2609,0,0,0,0,...,22,255,1.0,0.0,0.05,0.05,0.0,0.0,0.0,0.0
366408,0,b'tcp',b'private',b'S0',0,0,0,0,0,0,...,255,7,0.03,0.05,0.0,0.0,1.0,1.0,0.0,0.0
106319,0,b'tcp',b'http',b'SF',361,263,0,0,0,0,...,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
279055,0,b'icmp',b'ecr_i',b'SF',1032,0,0,0,0,0,...,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
374077,0,b'tcp',b'private',b'S0',0,0,0,0,0,0,...,255,11,0.04,0.05,0.0,0.0,1.0,1.0,0.0,0.0
282314,0,b'icmp',b'ecr_i',b'SF',1032,0,0,0,0,0,...,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
309444,0,b'icmp',b'ecr_i',b'SF',1032,0,0,0,0,0,...,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [10]:
# Lock the spark dataframe
spark_X.persist()

23/08/25 13:59:32 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


DataFrame[duration: bigint, protocol_type: binary, service: binary, flag: binary, src_bytes: bigint, dst_bytes: bigint, land: bigint, wrong_fragment: bigint, urgent: bigint, hot: bigint, num_failed_logins: bigint, logged_in: bigint, num_compromised: bigint, root_shell: bigint, su_attempted: bigint, num_root: bigint, num_file_creations: bigint, num_shells: bigint, num_access_files: bigint, num_outbound_cmds: bigint, is_host_login: bigint, is_guest_login: bigint, count: bigint, srv_count: bigint, serror_rate: double, srv_serror_rate: double, rerror_rate: double, srv_rerror_rate: double, same_srv_rate: double, diff_srv_rate: double, srv_diff_host_rate: double, dst_host_count: bigint, dst_host_srv_count: bigint, dst_host_same_srv_rate: double, dst_host_diff_srv_rate: double, dst_host_same_src_port_rate: double, dst_host_srv_diff_host_rate: double, dst_host_serror_rate: double, dst_host_srv_serror_rate: double, dst_host_rerror_rate: double, dst_host_srv_rerror_rate: double]

In [11]:
# Check the number of partitions the DataFrame is divided into the three workers yet
spark_X.rdd.getNumPartitions()

3

In [12]:
# Define the number of rows and columns in the dataframe
n_rows = spark_X.count()
n_cols = len(spark_X.dtypes)

In [13]:
# To look at the schema of the dataframe
spark_X.printSchema()

root
 |-- duration: long (nullable = true)
 |-- protocol_type: binary (nullable = true)
 |-- service: binary (nullable = true)
 |-- flag: binary (nullable = true)
 |-- src_bytes: long (nullable = true)
 |-- dst_bytes: long (nullable = true)
 |-- land: long (nullable = true)
 |-- wrong_fragment: long (nullable = true)
 |-- urgent: long (nullable = true)
 |-- hot: long (nullable = true)
 |-- num_failed_logins: long (nullable = true)
 |-- logged_in: long (nullable = true)
 |-- num_compromised: long (nullable = true)
 |-- root_shell: long (nullable = true)
 |-- su_attempted: long (nullable = true)
 |-- num_root: long (nullable = true)
 |-- num_file_creations: long (nullable = true)
 |-- num_shells: long (nullable = true)
 |-- num_access_files: long (nullable = true)
 |-- num_outbound_cmds: long (nullable = true)
 |-- is_host_login: long (nullable = true)
 |-- is_guest_login: long (nullable = true)
 |-- count: long (nullable = true)
 |-- srv_count: long (nullable = true)
 |-- serror_rate: d

In [14]:
# Delete the non-numerical values
clean_X = spark_X.select('*').drop('protocol_type', 'service', 'flag')
clean_X.printSchema()

root
 |-- duration: long (nullable = true)
 |-- src_bytes: long (nullable = true)
 |-- dst_bytes: long (nullable = true)
 |-- land: long (nullable = true)
 |-- wrong_fragment: long (nullable = true)
 |-- urgent: long (nullable = true)
 |-- hot: long (nullable = true)
 |-- num_failed_logins: long (nullable = true)
 |-- logged_in: long (nullable = true)
 |-- num_compromised: long (nullable = true)
 |-- root_shell: long (nullable = true)
 |-- su_attempted: long (nullable = true)
 |-- num_root: long (nullable = true)
 |-- num_file_creations: long (nullable = true)
 |-- num_shells: long (nullable = true)
 |-- num_access_files: long (nullable = true)
 |-- num_outbound_cmds: long (nullable = true)
 |-- is_host_login: long (nullable = true)
 |-- is_guest_login: long (nullable = true)
 |-- count: long (nullable = true)
 |-- srv_count: long (nullable = true)
 |-- serror_rate: double (nullable = true)
 |-- srv_serror_rate: double (nullable = true)
 |-- rerror_rate: double (nullable = true)
 |-- s

In [15]:
# Normalize the numerical columns
from pyspark.sql.functions import mean
from pyspark.sql.functions import stddev
from pyspark.ml.feature import StandardScaler

# norm_X = clean_X.map(lambda x: (x[i]-mean(x[i]))/stddev(x[i]) for i in range(n_cols))
# norm_X = clean_X.StandardScaler

i = 0
means = np.zeros(len(clean_X.columns))
# Calculate mean and variance for each column
for column in clean_X.columns:
    means[i] = float(clean_X.select(mean(column)).head()[0])
    i = i+1

i = 0
devstd = np.zeros(len(clean_X.columns))
# Calculate mean and variance for each column
for column in clean_X.columns:
    devstd[i] = float(clean_X.select(stddev(column)).head()[0])
    i = i+1

# norm_X = clean_X
means, devstd

(array([3.82150e+01, 8.10858e+02, 5.61198e+02, 1.00000e-03, 0.00000e+00,
        0.00000e+00, 3.80000e-02, 0.00000e+00, 1.47000e-01, 6.00000e-03,
        0.00000e+00, 0.00000e+00, 9.00000e-03, 5.00000e-03, 0.00000e+00,
        1.00000e-03, 0.00000e+00, 0.00000e+00, 1.00000e-03, 3.30848e+02,
        2.92576e+02, 1.84060e-01, 1.84090e-01, 5.19400e-02, 5.31400e-02,
        7.89810e-01, 1.87100e-02, 2.75200e-02, 2.32906e+02, 1.88994e+02,
        7.54910e-01, 3.08700e-02, 6.02630e-01, 7.82000e-03, 1.84920e-01,
        1.84230e-01, 5.21700e-02, 5.20100e-02]),
 array([4.88216167e+02, 3.43799781e+03, 2.82147557e+03, 3.16227766e-02,
        0.00000000e+00, 0.00000000e+00, 8.95185655e-01, 0.00000000e+00,
        3.54283102e-01, 8.92859002e-02, 0.00000000e+00, 0.00000000e+00,
        2.84604989e-01, 1.58113883e-01, 0.00000000e+00, 3.16227766e-02,
        0.00000000e+00, 0.00000000e+00, 3.16227766e-02, 2.13708348e+02,
        2.46625510e+02, 3.87654217e-01, 3.87413812e-01, 2.21889773e-01,
        

## Step 1: Choose a random sample from the dataset
This is the required step to begin the algorithm (doesn't need to be parallelized, since it is a select task)

In [ ]:
# Choose the first sample randomly

# random_sample = X.loc[np.random.randint(0, len(X))]
# NB: doesn't work when using spark_X (loc is not defined)

# Select the random row
random_n = [np.random.randint(0, n_rows)]

# Define a function that is able to select the single row in the dataframe
def getrows(df, rownums=None): # copied from the internet
    return df.rdd.zipWithIndex().filter(lambda x: x[1] in rownums).map(lambda x: x[0])
random_sample = getrows(spark_X, random_n).collect()


In [ ]:
# Evaluate the total error for the first sample (useless in the end)
mask = list((type(spark_X.take(1)[0][i]) for i in range(n_cols)))
# mask = (type(spark_X.head()[i]) for i in range(41)) # this works in tha same way

# Found in internet the function:
spark_X.dtypes

In [ ]:
# Divide the columns based on their type

# To print the selected columns
spark_X[[item[0] for item in spark_X.dtypes if item[1].startswith('double')]].printSchema()
spark_X[[item[0] for item in spark_X.dtypes if item[1].startswith('bigint')]].printSchema()
spark_X[[item[0] for item in spark_X.dtypes if item[1].startswith('binary')]].printSchema()

# To control that it still has 100 rows
print(spark_X[[item[0] for item in spark_X.dtypes if item[1].startswith('double')]].count(),
      spark_X[[item[0] for item in spark_X.dtypes if item[1].startswith('bigint')]].count(),
      spark_X[[item[0] for item in spark_X.dtypes if item[1].startswith('binary')]].count())

# To check if the sum is 41
n_double = len(spark_X[[item[0] for item in spark_X.dtypes if item[1].startswith('double')]].columns)
n_bigint = len(spark_X[[item[0] for item in spark_X.dtypes if item[1].startswith('bigint')]].columns)
n_binary = len(spark_X[[item[0] for item in spark_X.dtypes if item[1].startswith('binary')]].columns)

print('double =', n_double, '\tbigint = ', n_bigint, '\tbinary =', n_binary, '\nTotal is ', n_double + n_bigint + n_binary)


## Step 2: Initial cost
Then we evaluate the cost function (sum of the squares after the first selection)

In [ ]:
# Take the name and type of the relevant columns
coltypes = spark_X[[item[0] for item in spark_X.dtypes if item[1].startswith('double')]].dtypes

In [ ]:
# And then make a list of the names
from pyspark.sql.functions import col
columns = list(col(coltypes[i][0]) for i in range(len(coltypes)))
colnames = list(coltypes[i][0] for i in range(len(coltypes)))

In [ ]:
# To check that the columns are correctly selected
spark_X.select(columns).printSchema()

In [ ]:
# Creating a new column with the initial calue of the error (to be updated throughout the program)
initial_cost = (spark_X.select(columns)
                       .withColumn('initial_cost', sum((col(colname)-random_sample[0][colname])**2 for colname in colnames))
               )

In [ ]:
# To check if the evaluation of the squared errors works (TO BE DELETED)
initial_cost.toPandas()


In [ ]:
# Evaluate the total error (that will define the number of for loops done in the search)

from pyspark.sql.functions import sum
# phi = initial_cost.select(sum(initial_cost.initial_cost)).show()

phi = initial_cost.select(sum(initial_cost.initial_cost)).head()[0]
print(phi)
# IS IT A GOOD VALUE?


## Step 3: updating loop

# Stop worker and master
Stop the running Spark context (sc) and Spark session (spark)

In [ ]:
# sc.stop()
# spark.stop()

Finally, use `docker compose down` to stop and clear all running containers.